In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import json
import time
import pandas as pd
import numpy as np

with open("secrets.json", "r") as f:
    secrets = json.load(f)

<IPython.core.display.Javascript object>

In [3]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2


CLIENT_ID = secrets["spotify_client_id"]
CLIENT_SECRET = secrets["spotify_client_secret"]

credentials = oauth2.SpotifyClientCredentials(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET
)

token = credentials.get_access_token()
sp = spotipy.Spotify(auth=token)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


<IPython.core.display.Javascript object>

In [4]:
df = pd.read_csv("data/gen_playlist_tracks_full.csv", index_col=0)
df.head()

,id,artist,title,album,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,4Oun2ylbjFKMPTiaSbbCih,Cardi B,WAP (feat. Megan Thee Stallion),WAP (feat. Megan Thee Stallion),92,0.935,0.454,1,-7.509,1,0.3750,0.0194,0.000000,0.0824,0.357,133.073,187541,4
1,2SAqBLGA283SUiwJ3xOUVI,Drake,Laugh Now Cry Later (feat. Lil Durk),Laugh Now Cry Later (feat. Lil Durk),83,0.761,0.518,0,-8.871,1,0.1340,0.2440,0.000035,0.1070,0.522,133.976,261493,4
2,3H7ihDc1dqLriiWXwsc2po,Topic,Breaking Me,Breaking Me,96,0.789,0.720,8,-5.652,0,0.2180,0.2230,0.000000,0.1290,0.664,122.031,166794,4
3,6UelLqGlWMcVH1E5c4H7lY,Harry Styles,Watermelon Sugar,Fine Line,97,0.548,0.816,0,-4.209,1,0.0465,0.1220,0.000000,0.3350,0.557,95.390,174000,4
4,7ytR5pFWmSjzHJIeQkgog4,DaBaby,ROCKSTAR (feat. Roddy Ricch),BLAME IT ON BABY,100,0.746,0.690,11,-7.956,1,0.1640,0.2470,0.000000,0.1010,0.497,89.977,181733,4


<IPython.core.display.Javascript object>

In [19]:
artist_idxs = df["artist"].drop_duplicates().index
unique_artist_song_ids = df.iloc[artist_idxs]["id"]
unique_artist_song_ids[:50].index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 18,
            19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 36, 37,
            38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 53, 54, 55],
           dtype='int64')

<IPython.core.display.Javascript object>

In [5]:
all_artist_ids = []
all_track_types = []

# can only request in batches of 50
for i in np.arange(0, df.shape[0], 50):
    song_batch = df["id"][i : i + 50]
    try:
        res = sp.tracks(song_batch.values)
    except:
        # handles timeout errors
        time.sleep(5.1)
        res = sp.tracks(song_batch.values)

    # iterate thriguh each track
    for track in res["tracks"]:

        # list to store individual artist ids
        artist_ids = []
        artists = track["artists"]

        # iterate through in case there are multiple artists
        for artist in artists:
            artist_ids.append(artist["id"])

        # convert it all into one string separated by commas
        id_string = ", ".join(artist_ids)

        all_artist_ids.append(id_string)

        # grabbing song type to check for playlist/show/etc.
        all_track_types.append(track["type"])

<IPython.core.display.Javascript object>

In [6]:
len(all_artist_ids)

13071

<IPython.core.display.Javascript object>

In [7]:
df["artist_ids"] = pd.Series(all_artist_ids)
df["type"] = pd.Series(all_track_types)

<IPython.core.display.Javascript object>

In [9]:
df.to_csv("data/gen_playlist_tracks_full.csv")

<IPython.core.display.Javascript object>

In [10]:
split_ids = []
for ids in df['artist_ids']:
    for x in ids.split(', '):
        split_ids.append(x)



<IPython.core.display.Javascript object>

In [11]:
pd.Series(split_ids).shape

(18064,)

<IPython.core.display.Javascript object>

In [12]:
unique_artist_ids = pd.Series(split_ids).drop_duplicates().values

<IPython.core.display.Javascript object>

In [13]:
res = sp.artists(unique_artist_ids[:50])

<IPython.core.display.Javascript object>

In [18]:
res["artists"][0]["genres"].join(", ")

AttributeError: 'list' object has no attribute 'join'

<IPython.core.display.Javascript object>

In [101]:
artist_dicts = []
for i in np.arange(0, unique_artist_ids.shape[0], 50):
    artist_batch = unique_artist_ids[i : i + 50]
    try:
        res = sp.artists(artist_batch)
    except:
        # handles timeout errors
        time.sleep(5.1)
        res = sp.artists(artist_batch)

    for artist in res["artists"]:
        artist_name = artist["name"]
        artist_id = artist["id"]
        followers = artist["followers"]["total"]
        pop_rating = artist["popularity"]
        genres = artist["genres"]

        artist_dict = {
            "artist_name": artist_name,
            "artist_id": artist_id,
            "followers": followers,
            "popularity": pop_rating,
            "genres": genres,
        }
        artist_dicts.append(artist_dict)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [104]:
artist_df = pd.DataFrame(artist_dicts)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [105]:
artist_df.to_csv("artist_info.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
print(json.dumps(res, indent=4, sort_keys=True))

{
    "artists": [
        {
            "external_urls": {
                "spotify": "https://open.spotify.com/artist/4kYSro6naA4h99UJvo89HB"
            },
            "followers": {
                "href": null,
                "total": 12798539
            },
            "genres": [
                "pop",
                "pop rap",
                "rap"
            ],
            "href": "https://api.spotify.com/v1/artists/4kYSro6naA4h99UJvo89HB",
            "id": "4kYSro6naA4h99UJvo89HB",
            "images": [
                {
                    "height": 640,
                    "url": "https://i.scdn.co/image/e9c7d7e446f19d445898eb362417542b134863af",
                    "width": 640
                },
                {
                    "height": 320,
                    "url": "https://i.scdn.co/image/99ce83f26fff46e9a17429f7149ee3ebaaf51aa5",
                    "width": 320
                },
                {
                    "height": 160,
                    "u

<IPython.core.display.Javascript object>